[Соревнование на Kaggle](https://www.kaggle.com/competitions/stat-plus-ml-2024)

In [56]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

In [57]:
data = pd.read_csv('C:/Users/user/Downloads/train_contest.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Downloads/train_contest.csv'

### Пробуем с навыками

Сразу преобразовываем столбец requirement, то есть выстаскиваем его их словаря внутри snippet

In [ ]:
import json
import numpy as np

def safe_json_load(x):
    try:
        return json.loads(x.replace("'", '"').strip()).get('requirement', np.nan)
    except (json.JSONDecodeError, TypeError):
        return np.nan

data['requirement'] = data['snippet'].apply(lambda x: safe_json_load(x) if x else np.nan)
data['requirement'].fillna("нет данных", inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_30508\4156757221.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['requirement'].fillna("нет данных", inplace=True)


In [ ]:
data.iloc[1]

premium                                                                 False
name                                                    IT-специалист/ техник
department                                                                NaN
has_test                                                                False
response_letter_required                                                False
area                        {'id': '1130', 'name': 'Братск', 'url': 'https...
type                                       {'id': 'open', 'name': 'Открытая'}
address                                                                   NaN
response_url                                                              NaN
sort_point_distance                                                       NaN
published_at                                         2022-05-19T04:34:12+0300
created_at                                           2022-05-19T04:34:12+0300
archived                                                        

In [ ]:
data = data.explode("snippet", ignore_index = True).apply(pd.Series)

In [ ]:
data.shape

(30072, 33)

In [ ]:
data_embedding = data[['requirement','mean_salary']].copy()

Грузим модель для эмбеддинга paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
from sentence_transformers import SentenceTransformer

# Загрузка модели
model_embedding = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

Вот, что получилось на примере 1 строки

In [ ]:
sentences = data['requirement'].iloc[1]
model_embedding.encode(sentences)

array([ 3.93556245e-02, -1.04617402e-01,  3.13004944e-04, -2.14820325e-01,
       -3.39268565e-01, -2.22871602e-01,  1.93644240e-01,  1.76220149e-01,
        3.89971100e-02,  7.52309114e-02, -1.82990022e-02,  1.53643847e-01,
        1.40702531e-01, -2.13567197e-01,  1.51570663e-01,  2.11778671e-01,
       -2.86870189e-02, -5.79053396e-03, -4.12223786e-02, -1.91983745e-01,
       -2.28289038e-01, -2.18780339e-01, -1.86736628e-01,  1.48512637e-02,
       -2.49744177e-01,  7.01598376e-02, -1.46249771e-01,  9.26990062e-02,
       -6.09904043e-02, -1.25564754e-01, -5.90692014e-02,  5.73128052e-02,
        2.96236128e-01,  1.03631660e-01, -3.11210781e-01,  2.84732133e-01,
       -9.85726267e-02, -1.05293663e-02, -1.49707779e-01, -1.70426890e-01,
       -1.72708333e-02, -8.00514594e-02,  1.00057401e-01,  5.84230246e-03,
       -5.46059683e-02, -3.26102190e-02, -1.07429475e-01,  6.77547902e-02,
       -2.70189136e-01, -5.90772741e-02, -1.26231343e-01,  1.08772293e-02,
        8.21008906e-02, -

In [ ]:
data['requirement'].iloc[0]

'нет данных'

## Векторизуем батчами, потому что так быстрее, столбец requirement

In [ ]:
# Векторизация с батчами
batch_size = 100  
requirements = data_embedding['requirement'].tolist()  
embeddings = []

for i in tqdm(range(0, len(requirements), batch_size), desc="Векторизация"):
    batch = requirements[i:i+batch_size]
    batch_embeddings = model_embedding.encode(batch)  # Векторизуем батч
    embeddings.extend(batch_embeddings)  # Добавляем в список

# Проверяем, что количество эмбеддингов совпадает с исходным числом строк
assert len(embeddings) == len(data_embedding), "Ошибка: число эмбеддингов не совпадает с числом строк!"

# Присваиваем обратно в DataFrame
data_embedding['requirement_embedding'] = embeddings

Векторизация:   0%|          | 0/301 [00:00<?, ?it/s]

Векторизация: 100%|██████████| 301/301 [10:29<00:00,  2.09s/it]


In [ ]:
# Преобразуем список эмбеддингов в массив numpy
embeddings_matrix = np.array(data_embedding['requirement_embedding'].tolist())

# Создаём новые колонки для каждого элемента эмбеддинга
embedding_df = pd.DataFrame(embeddings_matrix, columns=[f'emb_{i}' for i in range(embeddings_matrix.shape[1])])

# Добавляем новые колонки в DataFrame
data_embedding_1 = pd.concat([data_embedding, embedding_df], axis=1)

In [ ]:
data_embedding_1.drop(columns=['requirement', 'requirement_embedding'], inplace=True)

In [ ]:
data_embedding_1

,mean_salary,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,125000.0,-0.121617,0.150209,-0.092759,-0.088475,0.094178,-0.195473,-0.137071,0.015531,-0.075729,...,0.097476,0.021756,0.203772,0.081538,-0.066362,0.061762,0.178605,-0.128850,0.037746,-0.082782
1,50000.0,0.039356,-0.104617,0.000313,-0.214820,-0.339269,-0.222872,0.193644,0.176220,0.038997,...,-0.004933,0.092350,0.198923,0.025798,-0.017191,0.067500,-0.204926,0.240490,-0.028713,-0.059574
2,80000.0,-0.109984,-0.014323,0.012643,-0.120398,-0.346929,-0.153878,-0.001590,0.315034,0.122713,...,0.203044,0.218634,0.045561,-0.313987,-0.029632,-0.118398,-0.067193,0.217805,-0.118572,0.051357
3,120000.0,0.124778,0.189373,0.020079,-0.004904,-0.123339,-0.149590,-0.094744,0.100798,0.132442,...,0.075075,0.145273,0.078865,-0.162683,-0.294222,0.047598,0.343692,-0.024289,0.181981,0.072665
4,15000.0,0.032440,0.006159,-0.238973,-0.019810,-0.113443,-0.025982,0.084321,0.215896,0.059868,...,-0.055104,0.103422,0.148903,-0.107115,-0.124028,0.165236,0.159618,0.221276,0.113031,-0.117873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,190000.0,0.020976,0.127628,0.125305,-0.089327,-0.007051,-0.161318,-0.072165,-0.050646,0.066078,...,0.251536,0.261149,0.186157,-0.213604,-0.298940,0.178916,0.071935,-0.093710,0.114566,0.116249
30068,45000.0,0.130400,-0.003853,-0.074075,0.096389,-0.067974,-0.050855,-0.087659,-0.045924,0.013160,...,0.192488,0.195827,0.121774,-0.079526,-0.058163,0.178221,-0.043815,0.004579,0.045786,0.084547
30069,75000.0,-0.185018,-0.113135,-0.365903,0.001326,-0.287603,0.081781,-0.181849,0.069974,-0.042242,...,0.044682,0.121651,0.298175,-0.263393,0.370150,0.193274,-0.197683,0.163921,-0.188703,-0.070287
30070,30000.0,0.058712,-0.043089,-0.065832,-0.162378,-0.171780,0.011066,0.014256,0.020893,-0.018532,...,0.202971,0.059301,0.057168,0.084419,0.031767,0.251676,0.019354,-0.227234,-0.182659,0.076201


### Дальше обучаем CatBoost

In [ ]:
# Целевая переменная (например, средняя зарплата)
y = data_embedding_1['mean_salary']

# Фичи (все кроме целевой переменной)
X = data_embedding_1.drop(columns=['mean_salary'])

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Инициализация модели
catboost = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=100)

# Обучение модели
catboost.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)


0:	learn: 105302.9480766	test: 97004.9188997	best: 97004.9188997 (0)	total: 101ms	remaining: 50.3s
100:	learn: 85109.5814200	test: 84471.0942914	best: 84471.0942914 (100)	total: 4.58s	remaining: 18.1s
200:	learn: 77492.5508114	test: 81710.3738416	best: 81710.3738416 (200)	total: 8.95s	remaining: 13.3s
300:	learn: 73105.9809859	test: 79919.2878375	best: 79916.3320671 (299)	total: 13.2s	remaining: 8.73s
400:	learn: 69902.8747629	test: 78606.0960101	best: 78606.0960101 (400)	total: 17.3s	remaining: 4.28s
499:	learn: 67441.7589303	test: 77638.9402727	best: 77637.7386616 (498)	total: 21.3s	remaining: 0us

bestTest = 77637.73866
bestIteration = 498

Shrink model to first 499 iterations.


In [ ]:
from sklearn.metrics import mean_absolute_error

# Предсказания
y_pred = catboost.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 43830.46


## Ошибка большая буду пробовать столбец description

In [ ]:
import json
import  re
from bs4 import BeautifulSoup

def clean_html(text):
    if pd.isna(text):  # Если значение NaN, возвращаем пустую строку
        return ""
    return BeautifulSoup(text, "html.parser").get_text(separator=" ")

data_embedding['description_clean'] = data['description'].apply(clean_html)

In [ ]:
data_embedding = data_embedding[['mean_salary','description_clean']]

In [ ]:
# Векторизация с батчами
batch_size = 100  
requirements = data_embedding['description_clean'].tolist()  
embeddings = []

for i in tqdm(range(0, len(requirements), batch_size), desc="Векторизация"):
    batch = requirements[i:i+batch_size]
    batch_embeddings = model_embedding.encode(batch)  # Векторизуем батч
    embeddings.extend(batch_embeddings)  # Добавляем в список

# Проверяем, что количество эмбеддингов совпадает с исходным числом строк
assert len(embeddings) == len(data_embedding), "Ошибка: число эмбеддингов не совпадает с числом строк!"

# Присваиваем обратно в DataFrame
data_embedding['description_clean'] = embeddings

Векторизация:   0%|          | 0/301 [00:00<?, ?it/s]

Векторизация: 100%|██████████| 301/301 [34:50<00:00,  6.95s/it]    


In [ ]:
# Преобразуем список эмбеддингов в массив numpy
embeddings_matrix = np.array(data_embedding['description_clean'].tolist())

# Создаём новые колонки для каждого элемента эмбеддинга
embedding_df = pd.DataFrame(embeddings_matrix, columns=[f'emb_{i}' for i in range(embeddings_matrix.shape[1])])

# Добавляем новые колонки в DataFrame
data_embedding_new = pd.concat([data_embedding, embedding_df], axis=1)

data_embedding_new = data_embedding_new.drop(columns= ['description_clean'])

In [ ]:
data_embedding_new

,mean_salary,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,125000.0,0.239519,0.000949,-0.158462,-0.080122,-0.151280,-0.044829,-0.221009,-0.057699,-0.126366,...,0.020951,0.107651,-0.014750,-0.051985,-0.030877,0.051000,0.139360,-0.028326,0.019401,0.057404
1,50000.0,0.043983,-0.016884,-0.002619,-0.153382,-0.273965,-0.135990,0.128149,0.211749,0.035189,...,0.025125,0.214797,0.152982,-0.057785,0.012083,0.086201,-0.203502,0.192398,-0.059146,-0.058440
2,80000.0,-0.053685,0.007963,0.007664,-0.070208,-0.235024,-0.034192,-0.181403,0.138877,0.068510,...,0.148899,0.314622,0.086222,-0.128844,0.142767,0.073888,-0.089741,0.142897,-0.096196,0.052010
3,120000.0,0.014182,0.039265,-0.156411,-0.096211,-0.008758,0.065084,0.000792,0.069800,-0.011577,...,0.063742,0.123730,-0.014496,-0.091761,-0.064206,0.017492,-0.058725,-0.037421,0.013903,0.156200
4,15000.0,-0.073199,0.156994,-0.071708,0.058729,-0.045829,0.041398,0.032269,0.004873,0.032542,...,-0.066835,0.168837,0.233503,-0.185761,0.124172,0.024971,0.054055,0.013299,-0.031699,-0.067141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,190000.0,-0.058647,0.025573,0.059531,-0.016806,-0.025522,-0.082279,0.160577,0.020724,-0.062479,...,-0.040628,0.118773,0.102080,-0.016615,0.078800,-0.001758,-0.002420,0.086373,0.052626,0.033976
30068,45000.0,-0.085697,0.057695,-0.078094,0.011128,-0.070757,0.059657,0.075377,-0.054589,-0.053394,...,0.091144,0.371247,0.115949,-0.181015,0.082554,0.118594,-0.165290,-0.018805,0.124331,0.056570
30069,75000.0,-0.049353,-0.055070,-0.101513,-0.079524,-0.141753,0.037081,-0.119910,0.059102,-0.116582,...,0.008496,0.150979,0.160178,-0.054738,0.186768,0.062854,-0.108735,-0.060235,-0.117578,0.029072
30070,30000.0,-0.073364,-0.018360,-0.292782,-0.115900,-0.162162,0.090767,0.196988,0.046671,0.050446,...,-0.105528,0.386336,0.151282,0.011033,0.091120,0.162197,-0.226204,-0.005608,-0.035085,0.144970


In [ ]:
# Целевая переменная (например, средняя зарплата)
y = data_embedding_new['mean_salary']

# Фичи (все кроме целевой переменной)
X = data_embedding_new.drop(columns=['mean_salary'])

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
catboost = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=100)

# Обучение модели
catboost.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)


0:	learn: 104728.1388890	test: 96200.2315587	best: 96200.2315587 (0)	total: 242ms	remaining: 2m
100:	learn: 76336.7769138	test: 76058.7098642	best: 76058.7098642 (100)	total: 4.69s	remaining: 18.5s
200:	learn: 66778.9043691	test: 72712.1919700	best: 72712.1919700 (200)	total: 8.53s	remaining: 12.7s
300:	learn: 60773.2065401	test: 70761.6121331	best: 70761.6121331 (300)	total: 12.4s	remaining: 8.21s
400:	learn: 56147.3047626	test: 69311.0189450	best: 69311.0189450 (400)	total: 16.3s	remaining: 4.02s
499:	learn: 52492.3743184	test: 68350.0233578	best: 68340.6965625 (498)	total: 20.2s	remaining: 0us

bestTest = 68340.69656
bestIteration = 498

Shrink model to first 499 iterations.


In [ ]:
from sklearn.metrics import mean_absolute_error

# Предсказания
y_pred = catboost.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 43830.46


In [ ]:
catboost_v2 = CatBoostRegressor(
    iterations=1000,
    depth=8,
    learning_rate=0.05,
    l2_leaf_reg=3,
    min_data_in_leaf=20,
    subsample=0.8,
    loss_function='RMSE',
    eval_metric='RMSE'
)

catboost_v2.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

0:	learn: 105321.9340266	test: 96855.5626807	best: 96855.5626807 (0)	total: 258ms	remaining: 4m 17s
1:	learn: 104397.2915760	test: 95984.9050635	best: 95984.9050635 (1)	total: 451ms	remaining: 3m 44s
2:	learn: 103520.4103444	test: 95182.3903699	best: 95182.3903699 (2)	total: 695ms	remaining: 3m 50s
3:	learn: 102758.0550970	test: 94424.9280787	best: 94424.9280787 (3)	total: 880ms	remaining: 3m 39s
4:	learn: 102053.7123492	test: 93751.9243381	best: 93751.9243381 (4)	total: 1.05s	remaining: 3m 28s
5:	learn: 101563.1188254	test: 93335.3282734	best: 93335.3282734 (5)	total: 1.21s	remaining: 3m 20s
6:	learn: 100987.5884389	test: 92726.5704434	best: 92726.5704434 (6)	total: 1.37s	remaining: 3m 14s
7:	learn: 100347.1234261	test: 92202.2007683	best: 92202.2007683 (7)	total: 1.53s	remaining: 3m 10s
8:	learn: 99735.6737492	test: 91784.0598174	best: 91784.0598174 (8)	total: 1.7s	remaining: 3m 6s
9:	learn: 99091.6949368	test: 91275.6963903	best: 91275.6963903 (9)	total: 1.86s	remaining: 3m 4s
10:	l

In [ ]:
from sklearn.metrics import mean_absolute_error

# Предсказания
y_pred = catboost_v2.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 38142.32


In [ ]:
catboost_v3 = CatBoostRegressor(
    iterations=1500,
    depth=10,
    learning_rate=0.05,
    l2_leaf_reg=10,
    min_data_in_leaf=20,
    subsample=0.8,
    loss_function='MAE'
)

catboost_v3.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

0:	learn: 54736.7967774	test: 54173.1418286	best: 54173.1418286 (0)	total: 431ms	remaining: 10m 46s
1:	learn: 54129.1172345	test: 53671.4214098	best: 53671.4214098 (1)	total: 834ms	remaining: 10m 24s
2:	learn: 53560.3480031	test: 53199.0618655	best: 53199.0618655 (2)	total: 1.23s	remaining: 10m 11s
3:	learn: 53053.6473379	test: 52764.9350003	best: 52764.9350003 (3)	total: 1.61s	remaining: 10m
4:	learn: 52446.8877792	test: 52249.3862525	best: 52249.3862525 (4)	total: 1.98s	remaining: 9m 52s
5:	learn: 51867.7840436	test: 51805.1568289	best: 51805.1568289 (5)	total: 2.38s	remaining: 9m 53s
6:	learn: 51294.5023516	test: 51357.0783603	best: 51357.0783603 (6)	total: 2.76s	remaining: 9m 48s
7:	learn: 50851.2164493	test: 50972.0889782	best: 50972.0889782 (7)	total: 3.12s	remaining: 9m 42s
8:	learn: 50388.7555361	test: 50600.1052566	best: 50600.1052566 (8)	total: 3.46s	remaining: 9m 34s
9:	learn: 49946.2549680	test: 50262.8389436	best: 50262.8389436 (9)	total: 3.81s	remaining: 9m 28s
10:	learn:

In [ ]:
# Предсказания
y_pred = catboost_v3.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 37942.95


In [ ]:
catboost_v4 = CatBoostRegressor(
    iterations=2500,
    depth=10,
    learning_rate=0.05,
    l2_leaf_reg=10,
    min_data_in_leaf=20,
    subsample=0.8,
    loss_function='MAE'
)

catboost_v4.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

0:	learn: 54736.7967774	test: 54173.1418286	best: 54173.1418286 (0)	total: 344ms	remaining: 14m 20s
1:	learn: 54129.1172345	test: 53671.4214098	best: 53671.4214098 (1)	total: 691ms	remaining: 14m 23s
2:	learn: 53560.3480031	test: 53199.0618655	best: 53199.0618655 (2)	total: 1.05s	remaining: 14m 35s
3:	learn: 53053.6473379	test: 52764.9350003	best: 52764.9350003 (3)	total: 1.4s	remaining: 14m 31s
4:	learn: 52446.8877792	test: 52249.3862525	best: 52249.3862525 (4)	total: 1.75s	remaining: 14m 31s
5:	learn: 51867.7840436	test: 51805.1568289	best: 51805.1568289 (5)	total: 2.1s	remaining: 14m 33s
6:	learn: 51294.5023516	test: 51357.0783603	best: 51357.0783603 (6)	total: 2.45s	remaining: 14m 31s
7:	learn: 50851.2164493	test: 50972.0889782	best: 50972.0889782 (7)	total: 2.79s	remaining: 14m 29s
8:	learn: 50388.7555361	test: 50600.1052566	best: 50600.1052566 (8)	total: 3.13s	remaining: 14m 27s
9:	learn: 49946.2549680	test: 50262.8389436	best: 50262.8389436 (9)	total: 3.47s	remaining: 14m 25s
10

In [ ]:
catboost_v5 = CatBoostRegressor(
    iterations=2500,
    depth=12,
    learning_rate=0.05,
    l2_leaf_reg=10,
    min_data_in_leaf=20,
    subsample=0.8,
    loss_function='MAE'
)

catboost_v5.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

0:	learn: 54494.9778833	test: 54067.5594373	best: 54067.5594373 (0)	total: 3.06s	remaining: 2h 7m 23s
1:	learn: 53755.5208336	test: 53481.5214529	best: 53481.5214529 (1)	total: 6.11s	remaining: 2h 7m 5s
2:	learn: 52941.7829434	test: 52906.3137289	best: 52906.3137289 (2)	total: 19.4s	remaining: 4h 29m 7s
3:	learn: 52114.6100860	test: 52304.0810144	best: 52304.0810144 (3)	total: 34.2s	remaining: 5h 55m 15s
4:	learn: 51331.4636878	test: 51779.1376713	best: 51779.1376713 (4)	total: 46.3s	remaining: 6h 25m 22s
5:	learn: 50507.5823415	test: 51225.8687613	best: 51225.8687613 (5)	total: 58.6s	remaining: 6h 46m 12s
6:	learn: 49862.6433205	test: 50739.4230451	best: 50739.4230451 (6)	total: 1m 10s	remaining: 6h 59m 32s
7:	learn: 49161.1698687	test: 50284.0861844	best: 50284.0861844 (7)	total: 1m 22s	remaining: 7h 10m 1s
8:	learn: 48495.4980887	test: 49874.5306349	best: 49874.5306349 (8)	total: 1m 34s	remaining: 7h 13m 46s
9:	learn: 47835.2855378	test: 49476.3370779	best: 49476.3370779 (9)	total: 

### Делаем прогноз

In [ ]:
new_X = pd.read_csv('C:/Users/user/Downloads/for_prediction.csv')


new_X['description_clean'] = new_X['description'].apply(clean_html)
new_X_embedding = new_X[['description_clean']]

In [ ]:
# Векторизация с батчами
batch_size = 100  
requirements = new_X_embedding['description_clean'].tolist()  
embeddings = []

for i in tqdm(range(0, len(requirements), batch_size), desc="Векторизация"):
    batch = requirements[i:i+batch_size]
    batch_embeddings = model_embedding.encode(batch)  # Векторизуем батч
    embeddings.extend(batch_embeddings)  # Добавляем в список

# Проверяем, что количество эмбеддингов совпадает с исходным числом строк
assert len(embeddings) == len(new_X_embedding), "Ошибка: число эмбеддингов не совпадает с числом строк!"

# Присваиваем обратно в DataFrame
new_X_embedding['description_clean'] = embeddings

Векторизация: 100%|██████████| 76/76 [08:44<00:00,  6.90s/it]
C:\Users\user\AppData\Local\Temp\ipykernel_30508\1605540260.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_X_embedding['description_clean'] = embeddings


In [ ]:
# Преобразуем список эмбеддингов в массив numpy
embeddings_matrix = np.array(new_X_embedding['description_clean'].tolist())

# Создаём новые колонки для каждого элемента эмбеддинга
embedding_df = pd.DataFrame(embeddings_matrix, columns=[f'emb_{i}' for i in range(embeddings_matrix.shape[1])])

# Добавляем новые колонки в DataFrame
new_X_embedding = pd.concat([new_X_embedding, embedding_df], axis=1)

new_X_embedding = new_X_embedding.drop(columns= ['description_clean'])

In [ ]:
predictions = catboost.predict(new_X_embedding)

display("Прогноз зарплаты:", pd.DataFrame(predictions))

'Прогноз зарплаты:'

,0
0,78022.788593
1,51264.583004
2,61392.665293
3,126111.815179
4,69999.807818
...,...
7513,36068.739069
7514,74031.435456
7515,134345.423610
7516,97927.317381


In [58]:
pd.DataFrame(predictions).to_csv('C:/Users/user/Downloads/predictions.csv')